# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [11]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import numpy as np
import requests

# Import API key
from api_keys import geoapify_key


<module 'matplotlib.colors' from 'C:\\Users\\riley\\anaconda3\\envs\\dev_new\\lib\\site-packages\\matplotlib\\colors.py'>


In [22]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aasiaat,68.7098,-52.8699,-17.58,86,30,4.14,GL,1730235487
1,1,ushuaia,-54.8000,-68.3000,6.81,65,75,11.83,AR,1730235488
2,2,grytviken,-54.2811,-36.5092,0.24,77,68,4.04,GS,1730235489
3,3,mount isa,-20.7333,139.5000,25.87,29,4,2.57,AU,1730235490
4,4,edinburgh of the seven seas,-37.0676,-12.3116,13.95,79,3,8.48,SH,1730235491


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [53]:
%%capture --no-display

colors = [f'{np.random.randint(0, 0xFFFFFF):06x}' for i in range(len(city_data_df))]
print(colors)

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    hover_cols=['City'])

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_df = city_data_df[(city_data_df['Max Temp'] <= 30) & (city_data_df['Humidity'] <= 30)]

# Drop any rows with null values
ideal_df = ideal_df.dropna()
    
# Display sample data
ideal_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
3,3,mount isa,-20.7333,139.5000,25.87,29,4,2.57,AU,1730235490
50,50,tamanrasset,22.7850,5.5228,21.95,16,0,4.12,DZ,1730235544
55,55,ghat,24.9647,10.1728,23.31,23,2,1.92,LY,1730235550
104,104,bardai,11.6500,7.2333,24.06,30,100,3.06,NG,1730235609
126,126,de queen,34.0379,-94.3413,28.90,29,7,6.01,US,1730235634


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_df[['City','Country','Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
3,mount isa,AU,-20.7333,139.5000,29,
50,tamanrasset,DZ,22.7850,5.5228,16,
55,ghat,LY,24.9647,10.1728,23,
104,bardai,NG,11.6500,7.2333,30,
126,de queen,US,34.0379,-94.3413,29,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [51]:
# Set parameters to search for a hotel
radius = 10000

params = {
    'apiKey': geoapify_key,
    'categories': 'accommodation.hotel',
    'filter' : '',
    'bias': '',
    }

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = 'circle:' + ','.join([str(row['Lng']),str(row['Lat']),str(10000)])
    params["bias"] = 'proximity:' + ','.join([str(row['Lng']),str(row['Lat'])])

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url,params = params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()
print(hotel_df['Humidity'].to_list())

Starting hotel search
mount isa - nearest hotel: Ibis Styles
tamanrasset - nearest hotel: فندق الأمان
ghat - nearest hotel: فندق تاسيلي
nyala - nearest hotel: سوني سفاري
chandler - nearest hotel: Crowne Plaza - San Marcos Golf Resort
diffa - nearest hotel: UNHCR guest house
viedma - nearest hotel: Níjar
nagqu - nearest hotel: Naqu Hotel
ingeniero jacobacci - nearest hotel: Gran Hotel Argentino
illizi - nearest hotel: بوناقة
nguigmi - nearest hotel: Guest PAM
el fasher - nearest hotel: Noor Aleman
sierra grande - nearest hotel: Sierra Grande
alice springs - nearest hotel: Aurora Alice Springs
mulchen - nearest hotel: Hostal LaTerraza
calama - nearest hotel: Apart Hotel Geotel Calama
[29, 16, 23, 19, 28, 14, 16, 28, 8, 18, 16, 14, 11, 30, 29, 6]


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [64]:
%%capture --no-display
# Removing cities without hotels within 10,000 meter radius
hotel_df = hotel_df[(hotel_df['Hotel Name'] != 'No hotel found')]

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    s = 'Humidity',
    frame_width = 700,
    frame_height = 500,
    color = 'City',
    hover_cols=['City','Hotel Name','Country'])

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)